In [1]:
import cv2
import numpy as np
import os

cap = cv2.VideoCapture(0)
data_path = 'C:\\Users\\Tarun Luthra\\Dropbox\\Coding\\Machine Learning\\Face Detection\\'


In [2]:
# KNN Code
def dist(x1, x2) -> float:
    '''This function returns the Euclidean Distance'''
    print('in dist function')
    return np.sqrt(sum((x1-x2)**2))


def knn(X, Y, queryPoint, k=5):
    print('in knn function')
    vals = []
    m = X.shape[0]

    print('m=',m)

    for i in range(m/2):
        print('knn i=',i)
        d = dist(queryPoint, X[i])
        vals.append((d, Y[i]))

    print('loop done')

    # vals is list of tuples of (dist,y_label) type
    vals = sorted(vals)
    vals = vals[:k]

    vals = np.array(vals)

    newVals = np.unique(vals[:, 1], return_counts=True)

    index = newVals[1].argmax()
    pred = newVals[0][index]

    print('returning from knn')

    return pred
#####################



In [3]:
# Face Detection
face_cascade = cv2.CascadeClassifier(data_path + "haarcascade_frontalface_alt.xml")

skip = 0
face_data = []
labels = []
class_id = 0
names = {}


In [4]:
# Data Preparation
for fx in os.listdir(data_path):
    if fx.endswith('.npy'):
        names[class_id] = fx[:-4]
        print("Loaded " + fx)
        data_item = np.load(data_path+fx, allow_pickle=True)

        face_data.append(data_item)

        target = class_id * np.ones((data_item.shape[0],))

        class_id += 1
        labels.append(target)

face_dataset = np.concatenate(face_data, axis=0)
face_labels = np.concatenate(labels, axis=0).reshape((-1, 1))

print('Data prepared')
print('X-',face_dataset.shape)
print('Y-',face_labels.shape)


Loaded Tarun.npy
Data prepared
X- (11, 1)
Y- (11, 1)


In [ ]:
# Testing
while True:
    ret, frame = cap.read()

    if ret == False:
        continue

    faces = face_cascade.detectMultiScale(frame, 1.3, 5)
    if len(faces) == 0:
        continue

    facecount=0

    for face in faces:
        x, y, w, h = face
        print('facecount=',facecount)
        facecount+=1

        offset = 10
        face_section = frame[y - offset: y + h + offset, x - offset: x + w + offset]

        out = knn(face_dataset, face_labels, face_section.flatten())

        pred_name = names[int(out)]

        cv2.putText(frame, pred_name, (x, y - 10),cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2, cv2.LINE_AA)
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 255), 2)

    cv2.imshow("Faces ", frame)

    keyPressed = cv2.waitKey(1) & 0xFF
    if keyPressed == ord('q'):
        break


cap.release()
cv2.destroyAllWindows()
